# [Jupyter]MLP.ipynb
Description   : Multiple layer perceptron for regression<br>
다층 은닉층을 가진 퍼셉트론 신경망 구축을 위해 회귀에서 활용되던 기본 함수를 가져왔다.<br>

In [1]:
import os

# 경로 초기화
os.chdir(r"C:\Users\TitusChoi\Desktop\Library\CodeLion\AI")

In [2]:
import numpy as np
import csv

# Hyperparameter
np.random.seed(1024)
RND_MEAN = 0
RND_STD = 0.003
LEARNING_RATE = 0.003

In [3]:
# Main function
def mlp_exec(epochs = 100, mb_size = 100, report = 10, tr = 0.75): # 학습 횟수, 미니 배치 사이즈, 학습 리포트, 훈련 비율(데이터 전부 훈련시키지 않고 그 중 훈련 비율만큼 훈련, 나머지는 시험 비율이 된다.)
    multiple_load_dataset() # 데이터 셋 여는 함수
    init_model() # 가중치와 편향 초기화 함수
    train_and_test(epochs, mb_size, report, tr) # 학습 및 신경망 성능 테스트 함수

In [32]:
def init_model():
    global hidden_config
    if hidden_config is not None:
        print('은닉 계층 {}개의 다층 퍼셉트론이 작동되었습니다.'. \
              format(len(hidden_config)))
        init_model_hiddens()
    else:
        print('은닉 계층 1개의 다층 퍼셉트론이 작동되었습니다.')
        init_model_hidden()

In [5]:
def set_hidden(info):
    global hidden_cnt, hidden_config
    if isinstance(info, int):
        hidden_cnt = info
        hidden_config = None
    else:
        hidden_config = info

In [6]:
# Data load, one-hot encoding
def multiple_load_dataset():
    # Loading datasets
    with open('./datasets/abalone.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None) # 첫 번째 index 건너뛰고 none으로 반환
        rows = []
        for row in csvreader:
            rows.append(row)

    # Global Variable : 함수 구축 시 주요하게 사용되는 변수를 확인하기 위한 장점때문에 사용
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 10, 1 # 독립변수의 크기와 종속변수의 크기
    
    # Buffer
    data = np.zeros([len(rows), input_cnt + output_cnt])

    # One-hot encoding
    for index, row in enumerate(rows):
        if row[0] == 'I' : data[index, 0] = 1
        elif row[0] == 'M' : data[index, 1] = 1
        elif row[0] == 'F' : data[index, 2] = 1
        data[index, 3:] = row[1:]

In [7]:
# Train and Test / Result check
def train_and_test(epochs, mb_size, report, tr):
    steps = arrange_data(mb_size, tr)           # 반환하는 값은 미니배치가 몇 덩어리(스텝)으로 쪼개지는지?
    test_x, test_y = get_test_data()               # 테스트 데이터에 대한 독립변수와 종속변수를 얻어내는 함수

    # epochs는 외부 for문으로 돌림
    # 미니배치는 내부 for문으로 돌리기 때문에 시간복잡도 증가
    for epoch in range(epochs):
        losses, accuracies = [], [] # epochs당 손실함수와 정확도 for 평균
        for n in range(steps):
            train_x, train_y = get_train_data(mb_size, n) # 미니배치 사이즈와 스텝의 수만큼 개별값을 받아 학습데이터의 독립, 종속변수 반환
            loss, accuracy = run_train(train_x, train_y)
            losses.append(loss)
            accuracies.append(accuracy)
        
        if report > 0 and (epoch + 1) % report == 0:
            accuracy = run_test(test_x, test_y)
            print('Epoch {} : loss = {:5.3f}, accuracy = {:5.3f}, Test={:5.3f}'\
                .format(epoch + 1, np.mean(losses), np.mean(accuracies), accuracy))
    
    final_accuracy = run_test(test_x, test_y)
    print("\n 최종 테스트 결과 : final accuracy = {:5.3f}\n".format(final_accuracy))

In [8]:
# Datasets Shuffle -> mini batch size
def arrange_data(mb_size, tr):
    global data, shuffle_map, test_begin_index
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    # 미니 배치 스텝 수를 구하는 과정
    steps = int(data.shape[0] * tr) // mb_size

    # Search boundary line
    test_begin_index = steps * mb_size
    return steps

In [9]:
# Training Data
def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_index, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_index]) # 무작위로 특정 인덱스까지 무작위로 섞기
    train_data = data[shuffle_map[mb_size * nth : mb_size * (nth + 1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

In [10]:
# Test Data
def get_test_data():
    global data, shuffle_map, test_begin_index, output_cnt
    test_data = data[shuffle_map[test_begin_index:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

In [11]:
# Running Training
def run_train(x, y):
    output, aux_nn = forward_neuralnet(x)       # 신경망 연산, aux는 보조자료를 의미
    loss, aux_pp = forward_postproc(output, y)  # 신경망에 따른 손실함수 연산 과정('Mean Square Error'), aux 뒤 pp는 postprocessing
    
    # 학습 1단계(순전파 및 학습과정)
    accuracy = eval_accuracy(output, y)         

    # 학습 2단계
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)

    return loss, accuracy

In [12]:
# Running Test
def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

In [13]:
# Forward Propagation Postprocessing : 신경망의 순전파에 따른 Mean Square Error를 도출하는 연산 
def forward_postproc(output, y):
    # 편차(diff) -> 제곱 -> 손실평균(loss)
    diff = output - y
    square = np.square(diff)

    loss = np.mean(square)
    # 편차를 반환하는 이유는 역전파 수행시 필요함(aux_pp)
    return loss, diff

In [14]:
# Backpropagation Postprocessing : 신경망의 역전파에 따른 선행 연산
def backprop_postproc(G_loss, diff):
    shape = diff.shape # M, N 정보 들어가 있음.
    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = diff * 2
    g_diff_output = 1

    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff

    return G_output

In [15]:
# hidden layer 구성
def init_model_hidden():
    global pm_output, pm_hidden, input_cnt, output_cnt, hidden_cnt

    pm_hidden = allocate_param_pair([input_cnt, hidden_cnt]) # parameter 값들
    pm_output = allocate_param_pair([hidden_cnt, output_cnt])

In [16]:
def alloc_param_pair(shape):
    weight = np.random.normal(RND_MEAN, RND_STD, shape)
    bias = np.zeros(shape[-1])
    return {'w':weight, 'b':bias}

In [17]:
# hypermarameter 통해 weight와 bias 할당
def allocate_param_pair(shape):
    weight = np.random.normal(RND_MEAN, RND_STD, shape)
    # 편향은 항상 출력 퍼셉트론의 크기와 통일, -1을 넣어 출력 크기에 맞춰 형성
    bias = np.zeros(shape[-1])
    return {'w' : weight, 'b' : bias} # 딕셔너리 형태로 할당

In [18]:
def forward_neuralnet(x):
    global hidden_config
    if hidden_config is not None:
        return forward_neuralnet_hiddens(x)
    else:
        return forward_neuralnet_hidden(x)

In [19]:
# forward neural network : 하나의 은닉 계층만을 사용
def forward_neuralnet_hidden(x):
    global pm_output, pm_hidden
    
    hidden = relu(np.matmul(x, pm_hidden['w']) + pm_hidden['b'])
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    
    return output, [x, hidden]

In [20]:
# activation function
def relu(x):
    return np.maximum(x, 0)

In [21]:
# backpropagation neural network : 하나의 은닉 계층만을 사용, 은닉 파라미터 실제 갱신 과정
def backprop_neuralnet_hidden(G_output, aux):
    global pm_output, pm_hidden
    x, hidden = aux

    g_output_w_out = hidden.transpose()
    G_w_output = np.matmul(g_output_w_out, G_output)
    G_b_output = np.sum(G_output, axis = 0)

    g_output_hidden = pm_output['w'].transpose()
    G_hidden = np.matmul(G_output, g_output_hidden)

    # 출력 계층 파라미터 갱신
    pm_output['w'] -= LEARNING_RATE * G_w_output
    pm_output['b'] -= LEARNING_RATE * G_b_output

    G_hidden = G_hidden * relu_derv(hidden)

    g_hidden_w_hidden = x.transpose()

    # 은닉 계층의 파라미터 갱신
    G_w_hidden = np.matmul(g_hidden_w_hidden, G_hidden)
    G_b_hidden = np.sum(G_hidden, axis = 0)

    # 학습률을 반영한 은닉 계층의 파라미터 갱신
    pm_hidden['w'] -= LEARNING_RATE * G_w_hidden
    pm_hidden['b'] -= LEARNING_RATE * G_b_hidden

In [22]:
# for multiple layer perceptron
def init_model_hiddens(): 
    global pm_output, pm_hiddens, input_cnt, output_cnt, hidden_config
    
    pm_hiddens = []
    prev_cnt = input_cnt
    
    for hidden_cnt in hidden_config:
        pm_hiddens.append(alloc_param_pair([prev_cnt, hidden_cnt]))
        prev_cnt = hidden_cnt
    
    pm_output = alloc_param_pair([prev_cnt, output_cnt])

In [23]:
def backprop_neuralnet(G_output, hiddens):
    global hidden_config
    if hidden_config is not None:
        backprop_neuralnet_hiddens(G_output, hiddens)
    else:
        backprop_neuralnet_hidden(G_output, hiddens)

In [24]:
# for multiple layer perceptron
def forward_neuralnet_hiddens(x):
    global pm_output, pm_hiddens
    
    hidden = x
    hiddens = [x]
    
    for pm_hidden in pm_hiddens:
        hidden = relu(np.matmul(hidden, pm_hidden['w']) + pm_hidden['b'])
        hiddens.append(hidden)
        
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    
    return output, hiddens

In [25]:
# for multiple layer perceptron
def backprop_neuralnet_hiddens(G_output, aux):
    global pm_output, pm_hiddens

    hiddens = aux
    
    g_output_w_out = hiddens[-1].transpose()
    G_w_out = np.matmul(g_output_w_out, G_output)
    G_b_out = np.sum(G_output, axis=0)

    g_output_hidden = pm_output['w'].transpose() 
    G_hidden = np.matmul(G_output, g_output_hidden)

    pm_output['w'] -= LEARNING_RATE * G_w_out
    pm_output['b'] -= LEARNING_RATE * G_b_out
    
    for n in reversed(range(len(pm_hiddens))):
        G_hidden = G_hidden * relu_derv(hiddens[n+1])

        g_hidden_w_hid = hiddens[n].transpose()
        G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
        G_b_hid = np.sum(G_hidden, axis=0)
    
        g_hidden_hidden = pm_hiddens[n]['w'].transpose()
        G_hidden = np.matmul(G_hidden, g_hidden_hidden)

        pm_hiddens[n]['w'] -= LEARNING_RATE * G_w_hid
        pm_hiddens[n]['b'] -= LEARNING_RATE * G_b_hid

In [26]:
def relu_derv(x):
    return np.sign(x)

In [27]:
# Evaluating Accuracy
def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output-y) / y))
    return 1 - mdiff

In [33]:
# main 함수 동작 구간
if __name__ == "__main__":
    # 은닉 계층 한 개
    set_hidden(3)
    mlp_exec()

은닉 계층 1개의 다층 퍼셉트론이 작동되었습니다.
Epoch 10 : loss = 7.047, accuracy = 0.804, Test=0.801
Epoch 20 : loss = 6.929, accuracy = 0.805, Test=0.803
Epoch 30 : loss = 6.813, accuracy = 0.807, Test=0.804
Epoch 40 : loss = 6.690, accuracy = 0.810, Test=0.802
Epoch 50 : loss = 6.569, accuracy = 0.812, Test=0.808
Epoch 60 : loss = 6.422, accuracy = 0.813, Test=0.809
Epoch 70 : loss = 6.261, accuracy = 0.815, Test=0.814
Epoch 80 : loss = 6.081, accuracy = 0.818, Test=0.821
Epoch 90 : loss = 5.885, accuracy = 0.821, Test=0.824
Epoch 100 : loss = 5.694, accuracy = 0.825, Test=0.823

 최종 테스트 결과 : final accuracy = 0.823



In [34]:
    # 은닉 계층 2개
    set_hidden([3, 12])
    mlp_exec()

은닉 계층 2개의 다층 퍼셉트론이 작동되었습니다.
Epoch 10 : loss = 10.102, accuracy = 0.735, Test=0.719
Epoch 20 : loss = 9.064, accuracy = 0.742, Test=0.731
Epoch 30 : loss = 6.946, accuracy = 0.791, Test=0.784
Epoch 40 : loss = 6.697, accuracy = 0.801, Test=0.794
Epoch 50 : loss = 6.535, accuracy = 0.807, Test=0.795
Epoch 60 : loss = 6.325, accuracy = 0.810, Test=0.799
Epoch 70 : loss = 6.000, accuracy = 0.816, Test=0.804
Epoch 80 : loss = 5.524, accuracy = 0.823, Test=0.814
Epoch 90 : loss = 5.080, accuracy = 0.831, Test=0.837
Epoch 100 : loss = 4.876, accuracy = 0.835, Test=0.832

 최종 테스트 결과 : final accuracy = 0.832

